In [1]:
import keras
import numpy as np
import csv
import matplotlib.pyplot as plt
data_file_dir = './PSFpack01/'
data_file_name = 'data_labels.csv'
num_labels = 6

image_dir_list = []
image_list = []
label_list = [[] for i in range(num_labels)]

    
with open(data_file_dir + data_file_name) as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for i,row in enumerate(readCSV):
        if i > 0:
            image_dir_list.append(row[0])
            image_list.append(plt.imread(data_file_dir + row[0] ))
            for j in range(num_labels):
                label_list[j].append(float(row[j+1]))
                
label_pick = 0
data = np.zeros((len(image_list),image_list[0].shape[0],image_list[0].shape[1],1))
label = np.zeros((len(image_list),1))
for i,image in enumerate(image_list):
    data[i,:,:,:] = image.reshape((1001,1001,1))
    label[i,:] = label_list[label_pick][i]
    
print 'data loaded!!'

print 'image shape:',image_list[0].shape
                
            
        

Using TensorFlow backend.


data loaded!!
image shape: (1001, 1001)


## build network

In [2]:
from keras.layers import *
from keras.models import Model
from keras import metrics
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint

num_filters = 20
kernel_size = (3,3)

inpt = Input(shape=(1001,1001,1))
c_opt = Conv2D(num_filters,kernel_size)(inpt)
m_opt = MaxPooling2D()(c_opt)

c2_opt = Conv2D(num_filters,kernel_size)(m_opt)
m2_opt = MaxPooling2D()(c2_opt)

c3_opt = Conv2D(num_filters,kernel_size)(m2_opt)
m3_opt = MaxPooling2D()(c3_opt)

c4_opt = Conv2D(num_filters,kernel_size)(m3_opt)
m4_opt = MaxPooling2D()(c4_opt)



f_opt = Flatten()(m4_opt)

opt = Dense(1)(f_opt)


model = Model([inpt],opt)

opt=Adam(lr=0.001/2)

model.compile(loss='mean_squared_error', optimizer=opt,metrics=[metrics.mae])

callbacks_list = [ModelCheckpoint('test', monitor='loss', verbose=1, 
                                  save_best_only=0, mode='auto',     
                                  save_weights_only='True')]

model.summary()

model.fit(data,label,epochs=10,batch_size=100,validation_split = 0.2,
                  callbacks = callbacks_list)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1001, 1001, 1)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 999, 999, 20)      200       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 499, 499, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 497, 497, 20)      3620      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 248, 248, 20)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 246, 246, 20)      3620      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 123, 123, 20)      0         
__________

KeyboardInterrupt: 